In [10]:
docx_path = 'doc/VPA-DD_Nigeria CPA6_ver2_NeEr_AnUe_15102021.docx'

In [12]:
import docx

def extract_docx(path: str) -> str:
    doc = docx.Document(path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

text = extract_docx(docx_path)
with open('test.txt', 'w') as f:
    f.write(text)